In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

import sys
sys.path.append('../../code/gwas_analysis')
from gwas_plotting import qqplot

## Convert to plink2 format

In [2]:
! ls ../../data/tmp/plink/LASVAGRImputationPostMissFilt20220223

AGRTMPRSS9SanityCheckchr19.vcfs		    LASVGWASAGR.vcfs	var25b
Fall2013				    March2012		var5
H3					    merged_pre2016	Winter2013
July2014NG				    MergePrePost	Winter2014
July2014SL				    reGenotypedNov2013
LASVAGRImputationPostMissFilt20220223.vcfs  var25a


In [3]:
! gsutil -m cp ../../data/tmp/plink/LASVAGRImputationPostMissFilt20220223.vcfs/* gs://popgen-gwas/data/genotype/merge/Imputed_VCF_20220226/

Copying file://../../data/tmp/plink/LASVAGRImputationPostMissFilt20220223.vcfs/1.vcf.gz [Content-Type=text/vcard]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://../../data/tmp/plink/LASVAGRImputationPostMissFilt20220223.vcfs/1.vcf.gz.csi [Content-Type=application/octet-stream]...
Copying file://../../data/tmp/plink/LASVAGRImputationPostMissFilt2022022

In [15]:
import os
inglob = '../../data/tmp/plink/LASVAGRImputationPostMissFilt20220223.vcfs/%s.vcf.gz'
outdir = '../../data/tmp/plink/MergePrePost/imputed_plink2/'
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:
chroms = [str(x) for x in range(1, 23)] + ['X']
for chrom in chroms:
    fn = inglob % chrom
    outbase = os.path.join(outdir, 'OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr%s' % chrom)
    cmd = "plink2 --make-pgen --vcf {vcf} dosage=DS --extract-if-info INFO '>'= 0.80 --out {out} --mac 1".format(vcf=fn, out=outbase)
    print(cmd)
    !{cmd}

plink2 --make-pgen --vcf ../../data/tmp/plink/LASVAGRImputationPostMissFilt20220223.vcfs/1.vcf.gz dosage=DS --extract-if-info INFO '>'= 0.80 --out ../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr1 --mac 1
PLINK v2.00a3LM 64-bit Intel (1 Jul 2021)      www.cog-genomics.org/plink/2.0/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr1.log.
Options in effect:
  --extract-if-info INFO >= 0.80
  --mac 1
  --make-pgen
  --out ../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr1
  --vcf ../../data/tmp/plink/LASVAGRImputationPostMissFilt20220223.vcfs/1.vcf.gz dosage=DS

Start time: Sat Feb 26 19:51:42 2022
64422 MiB RAM detected; reserving 32211 MiB for main workspace.
Using up to 16 threads (change this with --threads).
--vcf: 7084113 variants scanned.
--vcf: 7077k v

In [16]:
chroms = ['X']
for chrom in chroms:
    fn = inglob % chrom
    outbase = os.path.join(outdir, 'OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr%s' % chrom)
    cmd = "plink2 --make-pgen --vcf {vcf} dosage=DS --extract-if-info INFO '>'= 0.80 --out {out} --mac 1".format(vcf=fn, out=outbase)
    print(cmd)
    !{cmd}

plink2 --make-pgen --vcf ../../data/tmp/plink/LASVAGRImputationPostMissFilt20220223.vcfs/X.vcf.gz dosage=DS --extract-if-info INFO '>'= 0.80 --out ../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chrX --mac 1
PLINK v2.00a3LM 64-bit Intel (1 Jul 2021)      www.cog-genomics.org/plink/2.0/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chrX.log.
Options in effect:
  --extract-if-info INFO >= 0.80
  --mac 1
  --make-pgen
  --out ../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chrX
  --vcf ../../data/tmp/plink/LASVAGRImputationPostMissFilt20220223.vcfs/X.vcf.gz dosage=DS

Start time: Sun Feb 27 19:56:09 2022
64422 MiB RAM detected; reserving 32211 MiB for main workspace.
Using up to 16 threads (change this with --threads).
--vcf: 3477204 variants scanned.
--vcf: 3473k v

In [17]:
totalvars = 0
for chrom in list(range(1, 23)) + ['X']:
    outpvarfn = os.path.join(outdir, 'OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr%s.pvar' % str(chrom))
    z = pd.read_csv(outpvarfn, sep='\t', skiprows=16)
    totalvars += z.shape[0]
    
totalvars

21148723

## Over-write phenotype value with batch and fill in SEX

In [18]:
phenofn = 'gs://popgen-gwas/data/metadata/Annotated_GenotypedData/preANDpost2016_Merged_Complete_Metadata_WithAge_20210818.tsv'
metadata = pd.read_csv(phenofn, sep='\t')
metadata.index = metadata['Full_IID']

group_fn = 'gs://popgen-gwas/data/metadata/Annotated_GenotypedData/OmniH3Merged_PrePost2016_AggregatedLASVPhenotypes_20210829.tsv'
groups = pd.read_csv(group_fn, sep='\t')
groups.index = groups['#IID']
ind = groups.index[groups['Post2016_NG_LASV_qPCR2xPosOrSeqPos'] | groups['Post2016_NG_LASV_SeqPos'] | groups['Pre2016_NG_LASV_SeqPos'] | groups['Pre2016_SL_LASV_AgORSeqPos'] | groups['Post2016_SL_LASV_AgPos']]
metadata['LASV_Case'] = 0
metadata.loc[ind, 'LASV_Case'] = 1
metadata['LASV_Case'].value_counts()

metadata.head().T

Full_IID,July2014NG_SM-6C4M9,July2014NG_SM-4XBS9,July2014NG_SM-6C4MA,July2014NG_SM-6C4MB,July2014NG_SM-4XBSM
Full_IID,July2014NG_SM-6C4M9,July2014NG_SM-4XBS9,July2014NG_SM-6C4MA,July2014NG_SM-6C4MB,July2014NG_SM-4XBSM
Genotype_Sex,1,2,1,2,2
ID_Reformat,SM-6C4M9,SM-4XBS9,SM-6C4MA,SM-6C4MB,SM-4XBSM
SampleType,SuspectedLassaNG,Pre2016ControlNG,SuspectedLassaNG,SuspectedLassaNG,Pre2016ControlNG
sex,1,2,1,2,2
Disease,Lassa,Lassa,Lassa,Lassa,Lassa
ISTH_SandR_Key,NaN,NaN,NaN,NaN,NaN
KGH_Survivor_Key,NaN,NaN,NaN,NaN,NaN
KGH_AcuteLassa_Key,NaN,NaN,NaN,NaN,NaN
KGH_AcuteEbola_Key,NaN,NaN,NaN,NaN,NaN


In [19]:
metadata['Batch'].value_counts()

1                     1598
2                     1440
Winter2013            1405
March2012              485
Fall2013               333
Winter2014             299
July2014NG             129
July2014SL             119
reGenotypedNov2013      20
Name: Batch, dtype: int64

In [20]:
psams = [x for x in sorted(glob.glob('../../data/tmp/plink/MergePrePost/imputed_plink2/*.psam')) if 'Deduped' not in x]
psams

['../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr1.psam',
 '../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr10.psam',
 '../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr11.psam',
 '../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr12.psam',
 '../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr13.psam',
 '../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr14.psam',
 '../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr15.psam',
 '../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr16.psam',
 '../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr17.psam',
 '../../data/tmp/plink/MergeP

In [21]:
for fn in psams:
    psam = pd.read_csv(fn, sep='\t')
    psam.index = psam['#IID'].values
    psam['Batch'] = '0'
    psam.loc[psam['#IID'].isin(metadata.index[metadata['Batch']=='1']), 'Batch'] = '1'
    psam.loc[psam['#IID'].isin(metadata.index[metadata['Batch']=='2']), 'Batch'] = '2'
    psam.loc[psam['#IID'].isin(metadata.index[metadata['Array']=='Omni_25M_A']), 'Batch'] = '25'
    psam.loc[psam['#IID'].isin(metadata.index[metadata['Array']=='Omni_25M_B']), 'Batch'] = '26'
    psam.loc[psam['#IID'].isin(metadata.index[metadata['Array']=='Omni_5M']), 'Batch'] = '50'
    psam['SEX'] = metadata.loc[psam['#IID'],'sex'].replace({'M':'1', 'F':'2'})
    psam.to_csv(fn, sep='\t', index=False, header=True)

## Deduplicate

In [22]:
dupfiltfn_fam = '../../data/tmp/plink/MergePrePost/OmniH3Merged_PrePost2016.20210720.Geno_1e-1.filtBatchvars20210720.DupsToExclude.txt'
dupfiltfn = '../../data/tmp/plink/MergePrePost/OmniH3Merged_PrePost2016.20210720.Geno_1e-1.filtBatchvars20210720.DupsToExclude.IIDonly.txt'

fam = pd.read_csv(dupfiltfn_fam, sep='\t')
fam[['1']].to_csv(dupfiltfn, sep='\t', index=False, header=False)

In [23]:
for fn in sorted(glob.glob('../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr*.psam')):
    inbase = fn.replace('.psam', '')
    outbase = inbase.replace('.INFO_8e-1', '.INFO_8e-1.Deduped')
    cmd = 'plink2 --make-pgen --pfile {inv} --out {outv} --remove {removefn}'.format(inv=inbase, outv=outbase, removefn=dupfiltfn)
    print(cmd)
    !{cmd}

plink2 --make-pgen --pfile ../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr1 --out ../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.Deduped.chr1 --remove ../../data/tmp/plink/MergePrePost/OmniH3Merged_PrePost2016.20210720.Geno_1e-1.filtBatchvars20210720.DupsToExclude.IIDonly.txt
PLINK v2.00a3LM 64-bit Intel (1 Jul 2021)      www.cog-genomics.org/plink/2.0/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.Deduped.chr1.log.
Options in effect:
  --make-pgen
  --out ../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.Deduped.chr1
  --pfile ../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr1
  --remove ../../data/tmp/plink/MergePrePost/OmniH3Merged_PrePost2016.20210720.Geno_1e-1.filt

In [25]:
idmap = []
for fn in sorted(glob.glob('../../data/tmp/plink/MergePrePost/imputed_plink2/OmniH3Merged_PrePost2016_20220226.INFO_8e-1.chr*.pvar')):
    pvar = pd.read_csv(fn, sep='\t', skiprows=16)
    pvar['POSID'] = pvar['#CHROM'].astype(str)+':' + pvar['POS'].astype(str)+'[b37]'+pvar['REF']+','+pvar['ALT']
    idmap.append(pvar[['POSID', 'ID']])
    
idmap = pd.concat(idmap, axis=0)


In [26]:
idmap.to_csv('../../data/tmp/plink/MergePrePost/imputed_plink2/varidmap.tsv', sep='\t', index=False)

In [ ]:
! gsutil -m rsync -r ../../data/tmp/plink/MergePrePost/imputed_plink2/ gs://popgen-gwas/data/genotype/merge/Imputed_Plink2/